## Rolling Disk

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
me.init_vprinting()

In [ ]:
psi, theta, phi, xc, yc = me.dynamicsymbols("psi, theta, phi, x_c, y_c")
omega1, omega2, omega3, uc, vc = me.dynamicsymbols("omega1, omega2, omega3, u_c, v_c")
psid, thetad, phid, xcd, ycd = me.dynamicsymbols("psi, theta, phi, x_c, y_c", 1)
m, r, g = symbols("m, r, g")

In [ ]:
A, B, C, D = symbols("A, B, C, D", cls=me.ReferenceFrame)
B.orient_axis(A, A.z, psi)
C.orient_axis(B, B.x, theta)
D.orient_axis(C, C.y, phi)

A_w_D = D.ang_vel_in(A).express(D)
A_w_D

In [ ]:
rot_kin = A_w_D - (omega1 * D.x + omega2 * D.y + omega3 * D.z)
rot_kin = rot_kin.to_matrix(D)
transl_kin = [xcd - uc, ycd - vc]
kdes = transl_kin + list(rot_kin)
display(*kdes)

In [ ]:
O = me.Point("O")
Cp = O.locatenew("Cp", xc * A.x + yc * A.y)
G = Cp.locatenew("G", r * C.z)

O.set_vel(A, 0)
Cp.set_vel(A, uc * A.x + vc * A.y)
D.set_ang_vel(A, omega1 * D.x + omega2 * D.y + omega3 * D.z)
G.set_vel(A, D.ang_vel_in(A) ^ G.pos_from(Cp))

In [ ]:
vC0 = Cp.vel(A) + (D.ang_vel_in(C) ^ Cp.pos_from(G))
fv = [vC0.dot(A.x), vC0.dot(A.y)]
display(*fv)

In [ ]:
disc_inertia = m * r ** 2 / 4 * me.inertia(D, 1, 2, 1)
disc = me.RigidBody("Disc", G, D, m, (disc_inertia, G))
bodies = [disc]
loads = [(G, -m * g * A.z)]

In [ ]:
kane = me.KanesMethod(
    A,
    q_ind=[psi, theta, phi, xc, yc], 
    u_ind=[omega1, omega2, omega3],
    u_dependent=[uc, vc],
    kd_eqs=kdes,
    velocity_constraints=fv
)
(fr, frstar) = kane.kanes_equations(bodies, loads)

In [ ]:
rhs = kane.rhs()
rhs.simplify()
rhs

### Extraction of Information

In [ ]:
kane.kindiffdict()[phid]

In [ ]:
sd = {
    g: 9.81, r: 0.5, m: 5,
    # initial condition for positions
    psi: 0, theta: pi / 20, phi: 0.9 * pi / 2, xc: 0, yc: 0,
    # initial condition for independent velocities
    omega1: 0.5, omega2: 2, omega3: 0.5
}
kindiffdict = {k: v.subs(sd).n() for k, v in kane.kindiffdict().items()}
kindiffdict

In [ ]:
uc_vc = solve(Matrix(fv).subs(kindiffdict).subs(sd), [uc, vc])
uc_vc

In [ ]:
sd.update(uc_vc)

In [ ]:
import numpy as np
from pydy.system import System

sys = System(kane)
sys.constants = {m: sd[m], r: sd[r], g: sd[g]}
for s in [m, r, g]:
    sd.pop(s)
sys.initial_conditions = sd

fps = 60
t0, tf = 0, 15
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)
results = sys.integrate()

### Animation with PyDy

In [ ]:
from pydy.viz import Scene, Cylinder, VisualizationFrame, Sphere, Plane

disc_geom = Cylinder(r / 10, r, name="disc", color="orange")
plane_geom = Plane(50, 50, name="ground plane", color="white")
center_of_mass_geom = Sphere(r / 10, color="black", name="G")
contact_point_geom = Sphere(r / 10, color="blue", name="CP")

# to make it easier to visualize the rolling, let's create
# two rods, one aligned with the positive x-direction of
# the disk, the other aligned with the positive z-direction.
cylx = Cylinder(r, r / 15, name="cylx", color="red")
cylz = Cylinder(r, r / 15, name="cylz", color="green")
cylx_frame, cylz_frame = symbols("R1, R2", cls=me.ReferenceFrame)
cylx_frame.orient_axis(D, D.z, -pi/2)
cylz_frame.orient_axis(D, D.x, -pi/2)

rod1 = me.Body('rod1', frame=cylx_frame)
rod2 = me.Body('rod2', frame=cylz_frame)
rod1.masscenter.set_pos(disc.masscenter, r/2 * D.x)
rod2.masscenter.set_pos(disc.masscenter, r/2 * D.z)

vf_disc = VisualizationFrame("c", D, G, disc_geom)
vf_center_of_mass = VisualizationFrame("s1", D, G, center_of_mass_geom)
vf_contact_point = VisualizationFrame("s1", A, Cp, contact_point_geom)
vf_rod1 = VisualizationFrame("vf r1", rod1.frame, rod1.masscenter, cylx)
vf_rod2 = VisualizationFrame("vf r2", rod2.frame, rod2.masscenter, cylz)
vf_plane = VisualizationFrame("asd", A, O, plane_geom)

# because PyDy's default orientation is y-axis up, we need to
# perform a rotation in order to get z-axis up
N = me.ReferenceFrame("N")
N.orient_axis(A, A.x, pi/2)
scene = Scene(
    N, O,
    vf_disc, vf_center_of_mass, vf_contact_point,
    vf_plane, vf_rod1, vf_rod2,
    system=sys)
scene.display_jupyter(axes_arrow_length=0.5)

### Animation with Matplotlib

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

mv, rv, gv = [float(sys.constants[t]) for t in [m, r, g]]
psiv, thetav, phiv, xcv, ycv = results[0, :5]

OG = lambdify([psi, theta, phi, xc, yc, r], G.pos_from(O).express(A).to_matrix(A))

# circle perimeter
angle = np.linspace(0, 2*np.pi, 100)
xd = rv * np.cos(angle)
zd = rv * np.sin(angle)
yd = np.zeros_like(zd)
disk_coords = np.stack([xd, yd, zd])

# disk surface
rr = np.linspace(0, rv, 2)
rr, aa = np.meshgrid(rr, angle)
xds = rr * np.cos(aa)
zds = rr * np.sin(aa)
yds = np.zeros_like(zds)
shape = xds.shape
disk_surf_coords = np.stack([xds.flatten(), yds.flatten(), zds.flatten()])

# rotation matrix from ground frame to disk frame
R_AD = A.dcm(D)
R_AD = lambdify([psi, theta, phi], R_AD)

new_disk_coords = (R_AD(psiv, thetav, phiv) @ disk_coords) + OG(psiv, thetav, phiv, xcv, ycv, rv)
new_disk_surf_coords = (R_AD(psiv, thetav, phiv) @ disk_surf_coords) + OG(psiv, thetav, phiv, xcv, ycv, rv)

fig = plt.figure()
ax = fig.add_subplot(projection="3d")
circle, = ax.plot(new_disk_coords[0, :], new_disk_coords[1, :], new_disk_coords[2, :], color="tab:blue")
# reference point fixed on the surface of the disk
ref_point = ax.scatter(new_disk_coords[0, 0], new_disk_coords[1, 0], new_disk_coords[2, 0], color="r")
trajectory, = ax.plot(results[0, 3], results[0, 4], 0, color="tab:orange")
disk_surface = ax.plot_surface(
    new_disk_surf_coords[0, :].reshape(shape),
    new_disk_surf_coords[1, :].reshape(shape),
    new_disk_surf_coords[2, :].reshape(shape),
    alpha=0.5, color="tab:blue"
)
l = 3
ax.axis([xcv-l, xcv+l, ycv-l, ycv+l])
ax.set_zlim(0, rv * 2.5)
ax.set_zticks([0, rv * 2.5])
ax.set_aspect("equal")
ax.set_xlabel("x")
ax.set_ylabel("y")

In [ ]:
import matplotlib
matplotlib.rcParams["animation.embed_limit"] = 200

def update(idx):
    global disk_surface
    psiv, thetav, phiv, xcv, ycv = results[idx, :5]
    # update disc position and orientation
    new_disk_coords = (R_AD(psiv, thetav, phiv) @ disk_coords) + OG(psiv, thetav, phiv, xcv, ycv, rv)
    circle.set_data_3d([new_disk_coords[0, :], new_disk_coords[1, :], new_disk_coords[2, :]])
    # update trajectory
    cur_x, cur_y, cur_z = trajectory.get_data_3d()
    cur_x = np.append(cur_x, xcv)
    cur_y = np.append(cur_y, ycv)
    cur_z = np.append(cur_z, 0)
    trajectory.set_data_3d([cur_x, cur_y, cur_z])
    # scatter needs at least 2 points for update to work
    points = np.stack([new_disk_coords[:, 0], new_disk_coords[:, 0]])
    ref_point._offsets3d = (points[:, 0], points[:, 1], points[:, 2])
    # disk surface
    new_disk_surf_coords = (R_AD(psiv, thetav, phiv) @ disk_surf_coords) + OG(psiv, thetav, phiv, xcv, ycv, rv)
    disk_surface.remove()
    disk_surface = ax.plot_surface(
        new_disk_surf_coords[0, :].reshape(shape),
        new_disk_surf_coords[1, :].reshape(shape),
        new_disk_surf_coords[2, :].reshape(shape),
        alpha=0.5, color="tab:blue"
    )
    ax.set_title("t = {:.2f} s".format(sys.times[idx]))

ani = FuncAnimation(fig, update, frames=len(sys.times))
HTML(ani.to_jshtml(fps=fps))